<font color='blue'>An autoregressive, or AR$(p)$, model is created by regressing a time series on its past values, its lags.

 The simplest form of an autoregressive model is an <font color='blue'>AR$(1)$ model, signifying using only one lag term.

A first order autocorrelation model like this for a time series $x_t$ is:
<font color='blue'>$x_t = b_0 + b_1 x_{t - 1} + \epsilon_t$ </font><br>
    where $x_{t - 1}$ represents the value of the time series at time $(t - 1)$ and $\epsilon_t$ is the error term. 


We can extend this to an <font color='blue'>AR$(p)$ model, denoted:
$x_t = b_0 + b_1 x_{t-1} + b_2 x_{t - 2} \ldots + b_p x_{t - p} + \epsilon_t$


<font color='blue'>For an AR model to function properly, we must require that the time series is covariance stationary. 

This means that it follows three conditions:

1. The <font color='blue'>expected value of the time series is constant and finite at all times, i.e. $E[y_t] = \mu$ and $\mu < \infty$ for all values of $t$.
2. The <font color='blue'>variance of the time series is constant and finite for all time periods.
3. The <font color='blue'>covariance of the time series with itself for a fixed number of periods in either the future or the past is constant and finite for all time periods, i.e
$
COV(y_t, y_{t - s}) = \lambda, \  |\lambda| < \infty, \text{ $\lambda$ constant}, \  t = 1, 2, \ \ldots, T; \  s = 0, \pm 1, \pm 2, \ldots, \pm T
$


If these conditions are not satisfied, our estimation results will not have real-world meaning. Our estimates for the parameters will be biased, making any tests that we try to form using the model invalid. 

<font color='blue'>Unfortunately, it can be a real pain to find a covariance-stationary time series in the wild in financial markets. 

<font color='blue'>There are ways, however to make a non-stationary time series stationary.</font> Once we have performed this transformation, we can build an autoregressive models under the above assumptions.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels.tsa as tsa
import matplotlib.pyplot as plt

# ensures experiment runs the same every time
np.random.seed(100)

# This function simluates an AR process, generating a new value based on historial values,
# autoregressive coefficients b1 ... bk, and some randomness.
def AR(b, X, mu, sigma):
    l = min(len(b)-1, len(X))
    b0 = b[0]
    
    return b0 + np.dot(b[1:l+1], X[-l:]) + np.random.normal(mu, sigma)

b = np.array([0, 0.8, 0.1, 0.05])
X = np.array([1])

mu = 0
sigma = 1

for i in range(10000):
    X = np.append(X, AR(b, X, mu, sigma))
    
plt.plot(X)
plt.xlabel('Time')
plt.ylabel('AR Series Value');

<font color='blue'>Autoregressive processes will tend to have more extreme values than data drawn from say a normal distribution.</font> This is because the value at each time point is influenced by recent values. 

<font color='blue'>If the series randomly jumps up, it is more likely to stay up than a non-autoregressive series. This is known as 'fat-tailledness' (fat-tailed distribution) because the extremes on the pdf will be fatter than in a normal distribution.

Much talk of tail risk in finance comes from the fact that tail events do occur and are hard to model due to their infrequent occurrence. 

<font color='blue'>If we have reason to suspect that a process is autoregressive, we should expect risk from extreme tail events and adjust accordingly.

<font color='blue'>AR models are just one of the sources of tail risk, so don't assume that because a series is non-AR, it does not have tail risk.

In [ ]:
def compare_tails_to_normal(X):
    # Define matrix to store comparisons
    A = np.zeros((2,4))    
    for k in range(4):             
        #stores tail probabilities of the sample series vs a normal series
        A[0, k] = len(X[X > (k + 1)]) / float(len(X)) # Estimate tails of X        
        A[1, k] = 1 - stats.norm.cdf(k + 1) # Compare to Gaussian distribution
    print 'Frequency of std events in X \n1: %s\t2: %s\t3: %s\t4: %s' % tuple(A[0])
    print 'Frequency of std events in a normal process \n1: %s\t2: %s\t3: %s\t4: %s' % tuple(A[1])
    return A

compare_tails_to_normal(X);


Because an AR process has a tail heavy and non-normal distribution of outcomes, <font color='blue'>estimates of variance on AR processes will be wrong. 

This is dangerous because variance is used to calculate many quantities in staistics, most importantly confidence intervals and p-values. 

<font color='blue'>Because the width of the confidence interval is often based on a variance estimate, we can no longer trust p-values that come from AR processes.

In [ ]:
def compute_unadjusted_interval(X):
    T = len(X)
    # Compute mu and sigma MLE
    mu = np.mean(X)
    sigma = np.std(X)
    # Compute the bounds using standard error
    lower = mu - 1.96 * (sigma/np.sqrt(T))
    upper = mu + 1.96 * (sigma/np.sqrt(T))
    return lower, upper

# We'll make a function that returns true when the computed bounds contain 0
def check_unadjusted_coverage(X):
    l, u = compute_unadjusted_interval(X)
    # Check to make sure l <= 0 <= u
    if l <= 0 and u >= 0:
        return True
    else:
        return False
    
def simululate_AR_process(b, T):
    X = np.array([1])

    mu = 0
    sigma = 1

    for i in range(T):
        X = np.append(X, AR(b, X, mu, sigma))
        
    return X

In [ ]:
trials = 1000
outcomes = np.zeros((trials, 1))

for i in range(trials):
    #note these are the same values we used to generate the initial AR array
    Z = simululate_AR_process(np.array([0, 0.8, 0.1, 0.05]), 100)
    if check_unadjusted_coverage(Z):
        # The internal contains 0, the true value
        outcomes[i] = 1
    else:
        outcomes[i] = 0

In [ ]:
np.sum(outcomes) / trials

<font color='blue'>Stationarity tests should usually catch AR behavior and let us know that estimates of variance will be wrong.

<font color='blue'>In practice it can be very difficult to accurately estimate variance on an AR series, but one attempt to do this is the Newey-West estimation. You can find information on it [here](https://en.wikipedia.org/wiki/Newey%E2%80%93West_estimator).

<font color='blue'>In order to determine the order, $p$, of an AR$(p)$ model, we look at the autocorrelations of the time series. </font>These are the correlations of the series with its past values. 

<font color='blue'>The $k$-th order autocorrelation is
$
\rho_k = \frac{COV(x_t, x_{t - k})}{\sigma_x^2} = \frac{E[(x_t - \mu)(x_{t - k} - \mu)}{\sigma_x^2} </font>
$.  Where $k$ represents the number of periods lagged. 

<font color='blue'>We cannot directly observe the autocorrelations so we estimate them as 
$
\hat{\rho}_k = \frac{\sum_{t = k + 1}^T[(x_t - \bar{x})(x_{t - k} - \bar{x})]}{\sum_{t = 1}^T (x_t - \bar{x})^2}
$

For our purposes, we can use <font color='blue'>a pair of tools called the autocorrelation function (ACF) and the partial autocorrelation function (PACF) in order to determine the order of our model. The PACF controls for shorter lags, unlike the ACF. 


In [ ]:
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
X = simululate_AR_process(np.array([0, 0.8, 0.1, 0.05]), 1000)

In [ ]:
# We'll choose 40 lags. This is a bit arbitrary, but you want to include all the lags you think might
# feasibly impact the current value.
nlags = 40
# Note, this will produce nlags + 1 values, as we include the autocorrelation of
# X[-1] with X[-1], which is trivially 1.
# The reason this is done is because that is the 0th spot in the array and corresponds
# to the 0th lag of X[(-1)-0].
X_acf = acf(X, nlags=nlags)
print 'Autocorrelations:\n' + str(X_acf) + '\n'
X_pacf = pacf(X, nlags=nlags)
print 'Partial Autocorrelations:\n' + str(X_pacf)

In [ ]:
plt.plot(X_acf, 'ro')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title("ACF");

In [ ]:
plt.plot(X_pacf, 'ro')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title("PACF");

<font color='blue'>The `acf` and `pacf` functions will return confidence intervals on all the autocorrelations.

In [ ]:
# We have to set a confidence level for our intervals, we choose the standard of 95%,
# corresponding with an alpha of 0.05.
X_acf, X_acf_confs = acf(X, nlags=nlags, alpha=0.05)
X_pacf, X_pacf_confs = pacf(X, nlags=nlags, alpha=0.05)

In [ ]:
def plot_acf(X_acf, X_acf_confs, title='ACF'):
    # The confidence intervals are returned by the functions as (lower, upper)
    # The plotting function needs them in the form (x-lower, upper-x)
    errorbars = np.ndarray((2, len(X_acf)))
    errorbars[0, :] = X_acf - X_acf_confs[:,0]
    errorbars[1, :] = X_acf_confs[:,1] - X_acf

    plt.plot(X_acf, 'ro')
    plt.errorbar(range(len(X_acf)), X_acf, yerr=errorbars, fmt='none', ecolor='gray', capthick=2)
    plt.xlabel('Lag')
    plt.ylabel('Autocorrelation')
    plt.title(title);
plot_acf(X_acf, X_acf_confs)

In [ ]:
plot_acf(X_pacf, X_pacf_confs, title='PACF')

<font color='blue'>In a real-world time series, we use these plots to determine the order of our model. We would then attempt to fit a model using a maximum likelihood function.

In [ ]:
# Construct an unfitted model
model = tsa.api.AR(X)
# Fit it
model = model.fit()

In [ ]:
print 'Parameters'
print model.params
print 'Standard Error'
print model.bse

In [ ]:
# To plot this we'll need to format a confidence interval 2D array like the previous functions returned
# Here is some quick code to do that
model_confs = np.asarray((model.params - model.bse, model.params + model.bse)).T

plot_acf(model.params, model_confs, title='Model Estimated Parameters')

<font color='blue'>The reason that AR models will estimate many more lags than is actually the case is due to indirect dependency.</font> If $X_t$ depends on $X_{t-1}$, then indirectly and to a lesser extent it will depend on $X_{t-2}$. 

<font color='blue'>In the presence of more than one lag in the data generating process, we will get potentially complex harmonic structures in the lags.</font> These indirect dependencies will be picked up by a simple estimation.

In general it's rarely the case that you can get anything useful out of a model with many parameters.

In this case <font color='blue'>we want to select a number of lags that we believe explains what is happening, but without overfitting and choosing a model with way too many lags. 

<font color='blue'>We will use information criterion, specifically Akaike Information Criterion (AIC) and Bayes Information Criterion (BIC) to decide the correct number of parameters.

Interpreting the AIC and BIC is done as follows. <font color='blue'>Compute the AIC and BIC for all models we wish to consider, and note the smallest AIC and BIC recorded $AIC_{min}$ and $BIC_{min}$. 

These are the models which minimize information loss under each metric. 

<font color='blue'>For each type of IC We then can compute the *relative likelihood* of each model $i$ by taking 
$l = e^{(IC_{min} - IC_{i})/2}$

<font color='blue'>We can interpret $l$ as model $i$ is $l$ times as likely to minimize information loss, compared to the minimum AIC model.

In [ ]:
N = 10
AIC = np.zeros((N, 1))

for i in range(N):
    model = tsa.api.AR(X)
    model = model.fit(maxlag=(i+1))
    AIC[i] = model.aic
    
AIC_min = np.min(AIC)
model_min = np.argmin(AIC)

print 'Relative Likelihoods'
print np.exp((AIC_min-AIC) / 2)
print 'Number of parameters in minimum AIC model %s' % (model_min+1)

In [ ]:
N = 10
BIC = np.zeros((N, 1))

for i in range(N):
    model = tsa.api.AR(X)
    model = model.fit(maxlag=(i+1))
    BIC[i] = model.bic
    
BIC_min = np.min(BIC)
model_min = np.argmin(BIC)

print 'Relative Likelihoods'
print np.exp((BIC_min-BIC) / 2)
print 'Number of parameters in minimum BIC model %s' % (model_min+1)

Don't assume that using this method will always get you the right answer.

<font color='blue'>One final step we might do before performing an out of sample test for this model would be to evaluate its residual behavior. 

The AIC and BIC already do this to an extent, effectively measuring how much information is left on the table (in the residuals) after the model has made its predictions.

In [ ]:
model = tsa.api.AR(X)
model = model.fit(maxlag=3)

from statsmodels.stats.stattools import jarque_bera

score, pvalue, _, _ = jarque_bera(model.resid)

if pvalue < 0.10:
    print 'We have reason to suspect the residuals are not normally distributed.'
else:
    print 'The residuals seem normally distributed.'